In [ ]:
import geopandas as gpd
import pandas as pd
import os, json, xmltodict, math
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib
import load_data_new_type as ld
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LinearSegmentedColormap
from pathlib import Path
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import imageio

In [ ]:
root_path = "C:\\Git\\eqreplay"

shapefile_path = os.path.join(root_path, "Source\\Shapefile")

#cwbxml
with open(root_path+"\\Data\\2022_03_23\\CWB-EEW111018901.xml", encoding="utf-8") as xml_file:
    data_dict = xmltodict.parse(xml_file.read())
    cwb_data = json.loads(json.dumps(data_dict, ensure_ascii=False).encode("utf-8").decode())
    Identifier = cwb_data["earthquake"]["identifier"]
    SchemaVer = cwb_data["earthquake"]["schemaVer"]
    Language = cwb_data["earthquake"]["language"]
    SenderName = cwb_data["earthquake"]["senderName"]
    SentTime = cwb_data["earthquake"]["sent"]
    Status = cwb_data["earthquake"]["status"]
    Msgtype = cwb_data["earthquake"]["msgType"]
    MsgNo = cwb_data["earthquake"]["msgNo"]
    Description = cwb_data["earthquake"]["description"]
    Origintime = cwb_data["earthquake"]["originTime"]
    Lat = float(cwb_data["earthquake"]["epicenter"]["epicenterLat"]["#text"])
    Lon = float(cwb_data["earthquake"]["epicenter"]["epicenterLon"]["#text"])
    Depth = cwb_data["earthquake"]["depth"]["#text"]
    Magnitude = cwb_data["earthquake"]["magnitude"]["magnitudeValue"]
    pgaAdj = cwb_data["earthquake"]["pgaAdj"]
    
    cwb_origin_time = datetime.strptime(Origintime.replace("T", " "), "%Y-%m-%d %H:%M:%S+08:00").timestamp() 
    replay_start_time = datetime.strptime(Origintime.replace("T", " "), "%Y-%m-%d %H:%M:%S+08:00").timestamp() - 3
    print(replay_start_time, cwb_origin_time)
    print(datetime.fromtimestamp(replay_start_time))

##Depth判斷變更傳播速度
if 30.0 >= float(Depth) > 0 :
    #10.0Km
    s_speed = 0.0023
    p_speed = 0.0048
elif 60.0 >= float(Depth) > 30.0:
    #20.0Km
    s_speed = 0.0021
    p_speed = 0.0046
elif 90.0 >= float(Depth) > 30.0:
    #20.0Km
    s_speed = 0.0019
    p_speed = 0.0044
elif 120.0 >= float(Depth) > 90.0:
    #20.0Km
    s_speed = 0.0017
    p_speed = 0.0042
        

In [ ]:
town = gpd.read_file(shapefile_path+"\TOWN_MOI_1091016.shp", encoding="utf-8")
county = gpd.read_file(shapefile_path+"\COUNTY_MOI_1090820.shp", encoding="utf-8")
crs = county.crs

# variables #
#data
for r, d, f in os.walk(f"C:\\Git\\eqreplay\\Data\\2022_03_23"):
    print(f)
mqtt_data = f"C:\\Git\\eqreplay\\Data\\2022_03_23\\Tans_2022-3-23_Alarm.json"
site_data = f"C:\\Git\\eqreplay\\Data\\2022_03_23\\Tans_2022-3-23_Site.json"
trigger_site = []
trigger_cwb = []
calculating_list = []
data_list = []
sp_site = ["S00049"]

frame_rate = 0.1 #10 frame/per second
s_radius = 0.045 
p_radius = 0.055

#output
#Image_path = os.path.join(root_path, "Output\\Images\\"+Identifier)
#Gif_path = os.path.join(root_path, "Output\\Gifs\\"+Identifier)
# Path(Image_path).mkdir(parents=True, exist_ok=True)
# Path(Gif_path).mkdir(parents=True, exist_ok=True)

#zero point
origin_x = 119
origin_y = 21.8
delta_x = 3.3
delta_y = 3.7

t_size = 20 #text size

cbw = 2.3 #county_boundary_width
tbw = 0.5 #town_boundary_width

#eq_data
data = ld.load_data(mqtt_data)
replay_end_time = datetime.strptime(data[-1]["Date"]+" "+data[-1]["Time"], "%Y-%m-%d %H:%M:%S.%f").timestamp() #data.iloc[-1, 0].timestamp()
replay_loop = round((replay_end_time-replay_start_time)/frame_rate)
print(replay_end_time)


#site_data
site_df = ld.load_site(site_data)
print(type(site_data))
site_df["Calculate_Time"] = ""
print(site_df)

for index, row in site_df.iterrows():
    if row["Site_ID"] in sp_site:
        time_tmp = row["Trigger_Time"] - timedelta(seconds=1)
        site_df.iloc[index, site_df.columns.get_loc("Calculate_Time")] = time_tmp

    else:
        time_tmp = row["Trigger_Time"] - timedelta(seconds=3)
        site_df.iloc[index, site_df.columns.get_loc("Calculate_Time")] = time_tmp
#print(site_df)
site_geo = gpd.GeoDataFrame(site_df, geometry=gpd.points_from_xy(site_df.Site_Lon, site_df.Site_Lat))
# print(site_geo)

#for ind, row in site_geo.iterrows():
    #print(ind,row["Site_ID"])
#Datetime = list(set(site_df.Datetime.tolist()))
#Trigger_Time = list(set(site_df.Trigger_Time.tolist()))
#Datetime.sort()
#Trigger_Time.sort()

#site_times = {
#        "Datetime":Datetime,
#        "Trigger_Time":Trigger_Time
#}



In [1]:
#plotting eq data
color_map = [
        '#C4FBE2',
        '#26FF0B',
        '#FFFE0B',
        '#FF810B',
        '#FF550B',
        '#AC1F14',
        '#b834bf',
        '#cc6eff',
        #'#BB20D9'
]
cLevel = [1, 2, 3, 4, 5, 5.5, 6, 6.5, 7]
cmap, norm = matplotlib.colors.from_levels_and_colors(cLevel, color_map)#, extend="max")

#norm = matplotlib.colors.BoundaryNorm(cLevel,9)
#norm = matplotlib.colors.Normalize(vmin=1, vmax=7, clip=True)
#cm = matplotlib.colors.ListedColormap(color_map)
#cmap1 = LinearSegmentedColormap.from_list("my_colormap", color_map, N=9, gamma=1.0)


#map init
f, axes = plt.subplots(figsize=(15,15))


axes_cb = inset_axes(axes,
                width="3%",  # width = 50% of parent_bbox width
                height="30%",  # height : 5%
                loc='lower right')

f.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), cax=axes_cb, ax=axes, orientation='vertical', label='Intensity')

axes.set_title(Identifier, fontsize=25)
county.boundary.plot(ax=axes, color="black", edgecolor="black",linewidth=cbw, zorder=2)  #draw counties
l1 = town.plot(ax=axes, color="#dddddd", edgecolor="black", linewidth=tbw, zorder=1)     #draw towns
    
#set pic location
minx, miny, maxx, maxy = (origin_x, origin_y, origin_x + delta_x, origin_y + delta_y)
l1.set_xlim(minx, maxx)
l1.set_ylim(miny, maxy)
#event_time = (datetime.strptime(Origintime.replace("T", " "), "%Y-%m-%d %H:%M:%S+08:00"))
replay_time = datetime.fromtimestamp(replay_start_time)

#TextBoxes
axes_time = axes.text(minx+0.1, maxy-0.1, replay_time, size=t_size)
axes_site_count = axes.text(119.5, 24.7, "#Sites: "+str(len(trigger_site)), size=t_size, style='italic', c="gray")
axes_cwb_count = axes.text(119.5, 24.6, "#CWB: "+str(len(trigger_cwb)), size=t_size, style='italic', c="gray")


#plotting eq_center
eq_center = pd.DataFrame({"Name":"eq_center", "Latitude":[Lat], "Longitude":[Lon]})
eq_center = gpd.GeoDataFrame(eq_center, geometry=gpd.points_from_xy(eq_center.Longitude, eq_center.Latitude))
eq_center.plot(ax=axes, color="#00ffaa", marker="*", markersize=1000, zorder=3)

#before_event_process
while replay_time <= datetime.fromtimestamp(cwb_origin_time):
    replay_time += timedelta(seconds=frame_rate)
    print(replay_time)
    replay_time_str = replay_time.strftime("%Y-%m-%d %H_%M_%S.%f")[:-5]
    axes.texts[-3].set_text((replay_time_str.replace("_",":")+ "({})".format(round(replay_time.timestamp() - cwb_origin_time ,2))))
    axes.plot()
    axes.figure.savefig(Image_path+"\\"+replay_time_str+".png")
    
data_index = 0

print("------start main loop-------")
for time in range(replay_loop):
    
    #Main loop使用到的變數
    AreaDetail = []
    draw_area = gpd.GeoDataFrame()
    replay_time_str = replay_time.strftime("%Y-%m-%d %H_%M_%S.%f")[:-5]
    print(replay_time)
    
    #回放時間超過則停止
    if replay_time.timestamp() >= replay_end_time:
        break
    
    #P波、S波的初始化
    p_circle = plt.Circle((Lon, Lat), radius=p_radius, color="red", lw=2.0, ls="--", fill=False)
    axes.add_patch(p_circle)
    s_circle = plt.Circle((Lon, Lat), radius=s_radius, color="red", lw=2.0, fill=False)
    axes.add_patch(s_circle)
    
    #繪製計算中站台
    calculating_site = site_geo.loc[replay_time >= site_geo["Calculate_Time"]]
    for index, row in site_geo.iterrows():
        
        #若播放時間大於站台計算時間
        if replay_time >= row["Calculate_Time"]:
            if row["Site_ID"] not in calculating_list:
                calculating_list.append(row["Site_ID"])                
                calculating_site_plot = site_geo.iloc[[index], :]
                print(calculating_site_plot)
                calculating_site_plot.plot(ax=axes, color="#3BFF3A", edgecolor="black", marker=7, markersize=250, zorder=3)
                #axes.figure.savefig(str(time)+"C"+".png")

                
        #若播放時間大於站台觸發時間
        if replay_time >= row["Trigger_Time"] and row["Site_ID"] in calculating_list:
            if row["Site_ID"] not in trigger_site:
                trigger_site.append(row["Site_ID"])
                trigger_site_plot = site_geo.iloc[[index], :]
                trigger_site_plot.plot(ax=axes, color="#FFFB00", edgecolor="black", marker=7, markersize=250, zorder=3)
                axes.texts[-2].set_text("#Sites: "+str(len(trigger_site)))
        
    #若播放時間大於現地預警時間
    if replay_time >= datetime.strptime(data[data_index]["Date"]+" "+data[data_index]["Time"], "%Y-%m-%d %H:%M:%S.%f"):
        print("plot data")
        
        Areas = data[data_index]["Areas"]
        for Area in Areas:
            AreaDetail.append((Area["AreaCode"], Area["Source"], Area["PGAx"], ld.pga_to_level(float(Area["PGAx"]))))
            if Area["Source"] not in trigger_cwb and Area["Source"].startswith("CWB"):
                trigger_cwb.append(Area["Source"])
                axes.texts[-1].set_text("#CWB: "+str(len(trigger_cwb)))
                
        Area_df = pd.DataFrame(AreaDetail, columns=["AreaCode", "Source", "PGA", "Intensity"])
        Area_df = Area_df.sort_values(by=["AreaCode", "PGA"])
        for i in range(len(Area_df)):
            draw = town[town.TOWNCODE == Area_df.loc[i, "AreaCode"]]
            draw = draw.assign(Intensity = Area_df.loc[i, "Intensity"])
            draw_area = draw_area.append(draw)
        
        print(draw_area)
        draw_area.plot("Intensity", ax=axes, cmap=cmap, norm=norm)
        
        data_index += 1
        
    #P波、S波及字串處理
    ##1.P波、S波的累加
    p_radius += p_speed
    s_radius += s_speed
    ##2.時間字串處理
    axes.texts[-3].set_text((replay_time_str.replace("_",":")+ "({})".format(round(replay_time.timestamp() - cwb_origin_time ,2))))
    
    axes.plot()
    axes.figure.savefig(Image_path+"/"+replay_time_str + ".png")
    
    
    #儲存後的動作
    
    p_circle.remove()
    s_circle.remove()
    replay_time += timedelta(seconds=frame_rate)

NameError: name 'matplotlib' is not defined

In [ ]:
import imageio

impath = Image_path
print(impath)
filenames = os.listdir(impath)
img = []
for filename in filenames:
    #print(filename)
    img.append(imageio.imread(os.path.join(impath, filename)))
print("張數:",len(img))
imageio.mimsave(os.path.join(Gif_path, Identifier+".gif"), img)